<a href="https://colab.research.google.com/github/Andyfer004/Lab_6IA/blob/main/Lab6_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tasks 1 - Teoría


### 1. En un juego de suma cero para dos jugadores, ¿cómo funciona el algoritmo minimax para determinar la estrategia óptima para cada jugador? ¿Puede explicarnos el concepto de "valor minimax" y su importancia en este contexto?

El **algoritmo Minimax** se utiliza en juegos de suma cero de dos jugadores para encontrar la estrategia óptima maximizando la ganancia del jugador actual y minimizando la del oponente. Funciona de la siguiente manera:

1. Construye un árbol de juego con todas las posibles jugadas futuras.
2. Asigna valores a los nodos terminales según el resultado del juego.
3. Retrocede en el árbol aplicando:
   - **Maximización** en los turnos del jugador que busca ganar.
   - **Minimización** en los turnos del oponente.
4. La raíz del árbol recibe el **valor minimax**, que representa el mejor resultado posible considerando que el oponente juega óptimamente.

**Importancia del Valor Minimax:**
Es el valor óptimo que un jugador puede garantizarse asumiendo que el adversario jugará de la mejor forma posible.


### 2. Compare y contraste el algoritmo minimax con la poda alfa-beta. ¿Cómo mejora la poda alfa-beta la eficiencia del algoritmo minimax, particularmente en árboles de caza grandes? Proporcione un ejemplo para ilustrar la diferencia en la complejidad computacional entre la poda minimax y alfa-beta.

La **poda alfa-beta** es una optimización del algoritmo Minimax que reduce la cantidad de nodos evaluados sin alterar el resultado final. Se basa en los siguientes principios:

- **Alfa (α)**: Representa el mejor valor encontrado para el jugador Max hasta el momento.
- **Beta (β)**: Representa el mejor valor encontrado para el jugador Min hasta el momento.
- Si en algún nodo β ≤ α, no es necesario explorar más ramas, pues la jugada no será óptima.

**Ejemplo de complejidad computacional:**
- **Minimax sin poda:** Explora **O(b^d)** nodos (b = factor de ramificación, d = profundidad del árbol).
- **Minimax con poda alfa-beta:** En el mejor caso, reduce la complejidad a **O(b^(d/2))**, permitiendo evaluar el doble de profundidad en el mismo tiempo.


### 3. ¿Cuál es el papel de expectiminimax en juegos con incertidumbre, como aquellos que involucran nodos de azar o información oculta? ¿En qué se diferencia el expectiminimax del minimax en el manejo de resultados probabilísticos y cuáles son los desafíos clave que aborda?

El **expectiminimax** extiende minimax para juegos con **azar o información incompleta**, como dados o cartas ocultas. Funciona igual que minimax, pero introduce **nodos de azar**, donde se calcula un valor esperado en lugar de un valor mínimo o máximo:

- En los nodos del jugador, se selecciona la mejor acción.
- En los nodos del oponente, se selecciona la peor acción para el jugador.
- En los nodos de azar, se calcula la esperanza matemática ponderando los valores por sus probabilidades.

**Diferencias con Minimax:**
- En minimax, el resultado depende solo de los jugadores.
- En expectiminimax, se consideran distribuciones de probabilidad.

**Desafíos:**
- Mayor costo computacional debido a la expansión del árbol con nodos de azar.
- Requiere conocer o estimar correctamente las probabilidades de cada evento aleatorio.


